In [ ]:
# Complete EDA Notebook Implementation

# Cell 1: Setup and Imports
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.data.loader import DataLoader
from src.analysis.statistics import PortfolioAnalyzer
from src.visualization.plotter import InsuranceVisualizer
import warnings
warnings.filterwarnings('ignore')

# Cell 2: Load Data
loader = DataLoader('../data/raw/insurance_data.csv')
df = loader.load_data()

# Validate data
is_valid, validation_report = loader.validate_data()
print(f"Data Valid: {is_valid}")
print(f"Data Shape: {validation_report['rows']} rows, {validation_report['columns']} columns")

# Cell 3: Data Summarization
print("=== DESCRIPTIVE STATISTICS ===")
print(df.describe().T)

print("\n=== DATA TYPES ===")
print(df.dtypes)

print("\n=== MISSING VALUES ===")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({'Missing': missing, 'Percentage': missing_pct})
print(missing_df[missing_df['Missing'] > 0])

# Cell 4: Data Quality Assessment
# Convert date columns if present
date_columns = [col for col in df.columns if 'date' in col.lower() or 'Date' in col]
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Check for duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Cell 5: Univariate Analysis
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Univariate Analysis of Numerical Variables', fontsize=16)

# Histograms for numerical variables
num_cols = ['TotalPremium', 'TotalClaims', 'CustomValueEstimate']
for idx, col in enumerate(num_cols[:3]):
    ax = axes[idx // 3, idx % 3]
    df[col].hist(ax=ax, bins=30, edgecolor='black')
    ax.set_title(f'Distribution of {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Frequency')

# Box plots for outlier detection
for idx, col in enumerate(num_cols[:3]):
    ax = axes[idx // 3 + 1, idx % 3]
    df.boxplot(column=col, ax=ax)
    ax.set_title(f'Box Plot of {col}')

plt.tight_layout()
plt.show()

# Cell 6: Categorical Analysis
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
cat_cols = ['Province', 'VehicleType', 'Gender', 'VehicleMake'][:4]

for idx, col in enumerate(cat_cols):
    ax = axes[idx // 2, idx % 2]
    df[col].value_counts().head(10).plot(kind='bar', ax=ax, edgecolor='black')
    ax.set_title(f'Top 10 {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Count')
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Cell 7: Loss Ratio Analysis
analyzer = PortfolioAnalyzer(df)
loss_ratios = analyzer.calculate_loss_ratio()

print("=== LOSS RATIO ANALYSIS ===")
print(f"Overall Loss Ratio: {loss_ratios['overall_loss_ratio']:.4f}")

# Create loss ratio comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# By Province
province_series = pd.Series(loss_ratios['by_province'])
province_series.sort_values().plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Loss Ratio by Province')
axes[0].set_xlabel('Loss Ratio')

# By Vehicle Type
vehicle_series = pd.Series(loss_ratios['by_vehicle_type'])
vehicle_series.sort_values().plot(kind='barh', ax=axes[1], color='coral')
axes[1].set_title('Loss Ratio by Vehicle Type')
axes[1].set_xlabel('Loss Ratio')

# By Gender
gender_series = pd.Series(loss_ratios['by_gender'])
gender_series.plot(kind='bar', ax=axes[2], color=['lightblue', 'lightpink', 'lightgray'])
axes[2].set_title('Loss Ratio by Gender')
axes[2].set_xlabel('Gender')
axes[2].set_ylabel('Loss Ratio')

plt.tight_layout()
plt.show()

# Cell 8: Outlier Detection
print("=== OUTLIER DETECTION ===")
for col in ['TotalClaims', 'CustomValueEstimate', 'TotalPremium']:
    outliers, lower, upper = analyzer.detect_outliers_iqr(col)
    print(f"\n{col}:")
    print(f"  Lower bound: {lower:.2f}, Upper bound: {upper:.2f}")
    print(f"  Number of outliers: {len(outliers)}")
    print(f"  Percentage: {(len(outliers)/len(df))*100:.2f}%")
    if len(outliers) > 0:
        print(f"  Outlier range: {outliers.min():.2f} to {outliers.max():.2f}")

# Cell 9: Correlation Analysis
# Calculate correlations
correlation_matrix = df[['TotalPremium', 'TotalClaims', 'CustomValueEstimate']].corr()

# Create heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Key Financial Variables')
plt.tight_layout()
plt.show()

# Cell 10: Temporal Analysis (if date column exists)
if 'PolicyStartDate' in df.columns:
    df['MonthYear'] = df['PolicyStartDate'].dt.to_period('M')
    monthly_trends = df.groupby('MonthYear').agg({
        'TotalPremium': 'sum',
        'TotalClaims': 'sum'
    }).reset_index()
    monthly_trends['MonthYear'] = monthly_trends['MonthYear'].dt.to_timestamp()
    monthly_trends['LossRatio'] = monthly_trends['TotalClaims'] / monthly_trends['TotalPremium']
    
    fig, ax1 = plt.subplots(figsize=(12, 6))
    
    color1 = 'tab:blue'
    ax1.set_xlabel('Month')
    ax1.set_ylabel('Total Amount', color=color1)
    ax1.plot(monthly_trends['MonthYear'], monthly_trends['TotalPremium'], 
             label='Premium', color=color1, linewidth=2)
    ax1.plot(monthly_trends['MonthYear'], monthly_trends['TotalClaims'], 
             label='Claims', color='tab:red', linewidth=2)
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.legend(loc='upper left')
    
    ax2 = ax1.twinx()
    color2 = 'tab:green'
    ax2.set_ylabel('Loss Ratio', color=color2)
    ax2.plot(monthly_trends['MonthYear'], monthly_trends['LossRatio'], 
             label='Loss Ratio', color=color2, linewidth=2, linestyle='--')
    ax2.tick_params(axis='y', labelcolor=color2)
    
    plt.title('Monthly Trends: Premium, Claims, and Loss Ratio')
    fig.tight_layout()
    plt.show()

# Cell 11: CREATIVE VISUALIZATION 1 - Interactive Portfolio Dashboard
visualizer = InsuranceVisualizer(df)

# Create comprehensive dashboard
from IPython.display import display

print("=== CREATIVE VISUALIZATION 1: LOSS RATIO HEATMAP ===")
fig1 = visualizer.create_loss_ratio_heatmap()
fig1.show()

print("\n=== CREATIVE VISUALIZATION 2: PREMIUM VS CLAIMS SCATTER ===")
fig2 = visualizer.create_premium_claims_scatter()
fig2.show()

print("\n=== CREATIVE VISUALIZATION 3: VEHICLE MAKE ANALYSIS ===")
fig3 = visualizer.create_vehicle_make_analysis()
fig3.show()

# Cell 12: Statistical Distribution Analysis
print("=== STATISTICAL DISTRIBUTION ANALYSIS ===")
distributions = analyzer.analyze_distributions()

for col, stats in distributions.items():
    print(f"\n{col}:")
    print(f"  Normal Distribution: μ={stats['normal_params']['mu']:.2f}, σ={stats['normal_params']['sigma']:.2f}")
    print(f"  Normality Test p-value: {stats['normality_test']['p_value']:.4f}")
    print(f"  Is Normal Distribution: {stats['is_normal']}")

# Cell 13: Key Insights Summary
print("=== KEY INSIGHTS SUMMARY ===")

# Insight 1: Overall Portfolio Health
print("\n1. PORTFOLIO HEALTH:")
print(f"   - Overall Loss Ratio: {loss_ratios['overall_loss_ratio']:.2%}")
print(f"   - Average Premium: ${df['TotalPremium'].mean():,.2f}")
print(f"   - Average Claim: ${df['TotalClaims'].mean():,.2f}")

# Insight 2: Risk Factors
print("\n2. KEY RISK FACTORS:")
# Find province with highest loss ratio
highest_province = max(loss_ratios['by_province'].items(), key=lambda x: x[1])
lowest_province = min(loss_ratios['by_province'].items(), key=lambda x: x[1])
print(f"   - Highest Risk Province: {highest_province[0]} ({highest_province[1]:.2%})")
print(f"   - Lowest Risk Province: {lowest_province[0]} ({lowest_province[1]:.2%})")

# Insight 3: Vehicle Analysis
vehicle_analysis = df.groupby('VehicleType').agg({
    'TotalClaims': 'mean',
    'TotalPremium': 'mean'
}).round(2)
highest_claim_vehicle = vehicle_analysis['TotalClaims'].idxmax()
print(f"   - Vehicle Type with Highest Avg Claim: {highest_claim_vehicle}")

# Insight 4: Outlier Impact
total_outlier_claims = df.nlargest(10, 'TotalClaims')['TotalClaims'].sum()
total_all_claims = df['TotalClaims'].sum()
outlier_impact = (total_outlier_claims / total_all_claims) * 100
print(f"   - Top 10 claims represent {outlier_impact:.1f}% of total claims")

# Cell 14: Export Cleaned Data
df_cleaned = df.copy()
# Handle missing values (example: fill with median for numerical)
for col in ['TotalPremium', 'TotalClaims', 'CustomValueEstimate']:
    if df_cleaned[col].isnull().sum() > 0:
        df_cleaned[col].fillna(df_cleaned[col].median(), inplace=True)

# Save cleaned data
df_cleaned.to_csv('../data/processed/insurance_data_cleaned.csv', index=False)
print("Cleaned data saved to '../data/processed/insurance_data_cleaned.csv'")